In [ ]:
import gmsh
import random
import math

def create_random_spheres_in_cube(num_spheres, cube_size, sphere_radius):
    gmsh.initialize()
    gmsh.model.add("random_spheres")

    # Crear el cubo
    gmsh.model.occ.addBox(0, 0, 0, cube_size, cube_size, cube_size)

    # IDs de los volúmenes de las esferas
    sphere_volumes = []

    # Crear esferas en posiciones aleatorias
    for _ in range(num_spheres):
        x = random.uniform(sphere_radius, cube_size - sphere_radius)
        y = random.uniform(sphere_radius, cube_size - sphere_radius)
        z = random.uniform(sphere_radius, cube_size - sphere_radius)
        sphere = gmsh.model.occ.addSphere(x, y, z, sphere_radius)
        sphere_volumes.append(sphere)

    # Fusionar las esferas en un solo volumen
    gmsh.model.occ.synchronize()
    if len(sphere_volumes) > 1:
        _, combined_volume = gmsh.model.occ.fuse([(3, sphere_volumes[0])], [(3, vol) for vol in sphere_volumes[1:]])

    gmsh.model.occ.synchronize()
    gmsh.write("random_spheres.stp")

    # Crear la malla
    # mesh size 
    gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.01)
    gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.5)
    gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 20)

    gmsh.model.mesh.generate(3)
    gmsh.write("random_spheres.inp")
    gmsh.fltk.run()



    gmsh.finalize()

# Parámetros del modelo
num_spheres = 100  # Número de esferas
cube_size = 5   # Tamaño del cubo
sphere_radius = 0.5  # Radio de las esferas

create_random_spheres_in_cube(num_spheres, cube_size, sphere_radius)


In [ ]:
import gmsh
import random
import math

def check_distance_ok(x, y, z, spheres, min_dist):
    for sx, sy, sz in spheres:
        if math.sqrt((x - sx)**2 + (y - sy)**2 + (z - sz)**2) < min_dist:
            return False
    return True

def create_random_spheres_in_cube(num_spheres, cube_size, sphere_radius, threshold):
    gmsh.initialize()
    gmsh.model.add("random_spheres_adjusted")

    # Crear el cubo
    gmsh.model.occ.addBox(0, 0, 0, cube_size, cube_size, cube_size)

    # Lista para almacenar las posiciones de las esferas
    spheres = []
    sphere_volumes = []

    # Generar esferas en posiciones aleatorias respetando la distancia mínima
    attempts = 0
    while len(spheres) < num_spheres and attempts < 5000:
        x = random.uniform(sphere_radius, cube_size - sphere_radius)
        y = random.uniform(sphere_radius, cube_size - sphere_radius)
        z = random.uniform(sphere_radius, cube_size - sphere_radius)
        if check_distance_ok(x, y, z, spheres, 2 * sphere_radius + threshold):
            sphere = gmsh.model.occ.addSphere(x, y, z, sphere_radius)
            spheres.append((x, y, z))
            sphere_volumes.append(sphere)

        if attempts % 100 == 0:
            print("Attempt", attempts, "Number of spheres:", len(spheres))
        attempts += 1

    # Fusionar las esferas en un solo volumen
    gmsh.model.occ.synchronize()
    if len(sphere_volumes) > 1:
        _, combined_volume = gmsh.model.occ.fuse([(3, sphere_volumes[0])], [(3, vol) for vol in sphere_volumes[1:]])

    gmsh.model.occ.synchronize()
    gmsh.write("random_spheres_adjusted.stp")
    # mesh 
    #curve 
    gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.01)
    gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.5)
    gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 20)
    gmsh.model.mesh.generate(3)
    gmsh.write("random_spheres_adjusted.inp")
    gmsh.fltk.run()

    gmsh.finalize()

# Parámetros del modelo
num_spheres = 50   # Número de esferas
cube_size = 4    # Tamaño del cubo
sphere_radius = 0.5  # Radio de las esferas
threshold = -0.25   # Distancia mínima adicional entre las esferas

create_random_spheres_in_cube(num_spheres, cube_size, sphere_radius, threshold)


In [ ]:
import gmsh
import numpy as np

gmsh.initialize()
gmsh.model.add("rounded_square")

lc_x = 3.0  # Tamaño del cuadrado
lc_y = 1.0  # Tamaño del cuadrado
r = 0.2   # Radio de las esquinas redondeadas

# Definir los ocho puntos del borde recto del cuadrado


p =     [[-lc_x/2 + r  , -lc_y/2     , 0],
         [ lc_x/2 - r  , -lc_y/2     , 0],
         [ lc_x/2      , -lc_y/2 + r , 0],
         [ lc_x/2      ,  lc_y/2 - r , 0],
         [ lc_x/2 - r  ,  lc_y/2     , 0],
         [-lc_x/2 + r  ,  lc_y/2     , 0],
         [-lc_x/2      ,  lc_y/2 - r , 0],
         [-lc_x/2      , -lc_y/2 + r , 0]]


c =     [[ lc_x/2 - r  , -lc_y/2 + r  ,  0 ],  # Esquina inferior derecha
         [ lc_x/2 - r  ,  lc_y/2 - r  ,  0 ],   # Esquina superior derecha
         [-lc_x/2 + r  ,  lc_y/2 - r  ,  0 ],  # Esquina superior izquierda
         [-lc_x/2 + r  , -lc_y/2 + r  ,  0 ]] # Esquina inferior izquierda

p = np.array(p)
c = np.array(c)

# 
# #     Parámetros:
#     - x, y, z: Centro del rectángulo.
#     - zAxis: Vector normal al plano del rectángulo.
#     - xAxis: Vector que define la orientación del rectángulo.

# rotate 
# gmsh.model.occ.rotate([(2, surf)], 0, 0, 0, 
#                                    0, 1, 0, 90)


p1 = gmsh.model.occ.addPoint(p[0,0], p[0,1], p[0,2])
p2 = gmsh.model.occ.addPoint(p[1,0], p[1,1], p[1,2])
p3 = gmsh.model.occ.addPoint(p[2,0], p[2,1], p[2,2])
p4 = gmsh.model.occ.addPoint(p[3,0], p[3,1], p[3,2])
p5 = gmsh.model.occ.addPoint(p[4,0], p[4,1], p[4,2])
p6 = gmsh.model.occ.addPoint(p[5,0], p[5,1], p[5,2])
p7 = gmsh.model.occ.addPoint(p[6,0], p[6,1], p[6,2])
p8 = gmsh.model.occ.addPoint(p[7,0], p[7,1], p[7,2])

# Definir los centros de los arcos en las esquinas
c1 = gmsh.model.occ.addPoint(c[0,0], c[0,1], c[0,2])
c2 = gmsh.model.occ.addPoint(c[1,0], c[1,1], c[1,2])
c3 = gmsh.model.occ.addPoint(c[2,0], c[2,1], c[2,2])
c4 = gmsh.model.occ.addPoint(c[3,0], c[3,1], c[3,2])

# Definir los arcos
a1 = gmsh.model.occ.addCircleArc(p2, c1, p3)
a2 = gmsh.model.occ.addCircleArc(p4, c2, p5)
a3 = gmsh.model.occ.addCircleArc(p6, c3, p7)
a4 = gmsh.model.occ.addCircleArc(p8, c4, p1)

# Definir las líneas rectas
l1 = gmsh.model.occ.addLine(p1, p2)
l2 = gmsh.model.occ.addLine(p3, p4)
l3 = gmsh.model.occ.addLine(p5, p6)
l4 = gmsh.model.occ.addLine(p7, p8)

# Crear la curva cerrada
loop = gmsh.model.occ.addCurveLoop([l1, a1, l2, a2, l3, a3, l4, a4])

# Crear la superficie
surf = gmsh.model.occ.addPlaneSurface([loop])
gmsh.model.occ.synchronize()


gmsh.model.occ.synchronize()

# 


# Sincronizar y visualizar
gmsh.fltk.run()
gmsh.finalize()


In [ ]:
surf